In [9]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from solver import *
from cache import *
from input_preprocessing import *

In [2]:
device = torch.device('cpu')

x_grid=np.linspace(0,1,21)
t_grid=np.linspace(0,1,21)

x = torch.from_numpy(x_grid)
t = torch.from_numpy(t_grid)

grid = torch.cartesian_prod(x, t).float()

grid.to(device)


C:\Users\nikiniki\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\functional.py:1069: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.cartesian_prod(tensors)  # type: ignore[attr-defined]


tensor([[0.0000, 0.0000],
        [0.0000, 0.0500],
        [0.0000, 0.1000],
        [0.0000, 0.1500],
        [0.0000, 0.2000],
        [0.0000, 0.2500],
        [0.0000, 0.3000],
        [0.0000, 0.3500],
        [0.0000, 0.4000],
        [0.0000, 0.4500],
        [0.0000, 0.5000],
        [0.0000, 0.5500],
        [0.0000, 0.6000],
        [0.0000, 0.6500],
        [0.0000, 0.7000],
        [0.0000, 0.7500],
        [0.0000, 0.8000],
        [0.0000, 0.8500],
        [0.0000, 0.9000],
        [0.0000, 0.9500],
        [0.0000, 1.0000],
        [0.0500, 0.0000],
        [0.0500, 0.0500],
        [0.0500, 0.1000],
        [0.0500, 0.1500],
        [0.0500, 0.2000],
        [0.0500, 0.2500],
        [0.0500, 0.3000],
        [0.0500, 0.3500],
        [0.0500, 0.4000],
        [0.0500, 0.4500],
        [0.0500, 0.5000],
        [0.0500, 0.5500],
        [0.0500, 0.6000],
        [0.0500, 0.6500],
        [0.0500, 0.7000],
        [0.0500, 0.7500],
        [0.0500, 0.8000],
        [0.0

In [4]:
prepared_grid,grid_dict,point_type = grid_prepare(grid)

In [16]:
model = torch.nn.Sequential(
        torch.nn.Linear(2, 100),
        torch.nn.Tanh(),
        torch.nn.Linear(100, 100),
        torch.nn.Tanh(),
        torch.nn.Linear(100, 100),
        torch.nn.Tanh(),
        torch.nn.Linear(100, 2)
    )

In [13]:
model(prepared_grid).shape[1]

1

In [8]:
def solution_print(prepared_grid,model,title=None):
    if prepared_grid.shape[1] == 2:
        nvars_model = model(prepared_grid).shape[1]
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        if title!=None:
            ax.set_title(title)
        if nvars_model == 1:
            ax.plot_trisurf(prepared_grid[:, 0].reshape(-1), prepared_grid[:, 1].reshape(-1),
                        model(prepared_grid).detach().numpy().reshape(-1), cmap=cm.jet, linewidth=0.2, alpha=1)
        elif nvars_model == 2:
            ax.plot_trisurf(prepared_grid[:, 0].reshape(-1), prepared_grid[:, 1].reshape(-1),
                        model(prepared_grid)[:,0].detach().numpy().reshape(-1), cmap=cm.jet, linewidth=0.2, alpha=1)
            ax.plot_trisurf(prepared_grid[:, 0].reshape(-1), prepared_grid[:, 1].reshape(-1),
                        model(prepared_grid)[:,1].detach().numpy().reshape(-1), cmap=cm.jet, linewidth=0.2, alpha=1)
        ax.set_xlabel("x1")
        ax.set_ylabel("x2")
        plt.show()

    if prepared_grid.shape[1] == 1:
        fig = plt.figure()
        plt.scatter(prepared_grid.reshape(-1), model(prepared_grid).detach().numpy().reshape(-1))
        plt.show()

1

In [19]:
model(prepared_grid)[:,0].detach().numpy().reshape(-1)

array([0.40675732, 0.40516472, 0.4034188 , 0.40152574, 0.39949203,
       0.39732456, 0.3950305 , 0.3926173 , 0.3900926 , 0.3874644 ,
       0.38474068, 0.38192955, 0.37903938, 0.37607816, 0.37305424,
       0.36997566, 0.3668506 , 0.36368665, 0.36049163, 0.35727292,
       0.3540377 , 0.30036303, 0.2986026 , 0.29674968, 0.29481247,
       0.29279926, 0.29071853, 0.28857866, 0.286388  , 0.284155  ,
       0.28188783, 0.27959442, 0.2772827 , 0.27496025, 0.27263427,
       0.2703117 , 0.26799923, 0.26570293, 0.26342878, 0.26118204,
       0.25896782, 0.30769867, 0.31484905, 0.3218054 , 0.32856002,
       0.33510637, 0.34143877, 0.3475527 , 0.35344446, 0.3591117 ,
       0.36455265, 0.36976656, 0.37475362, 0.37951487, 0.3840521 ,
       0.38836762, 0.39246494, 0.39634782, 0.40002066, 0.40348867,
       0.3059327 , 0.3040684 , 0.30211404, 0.30007812, 0.29796895,
       0.29579514, 0.2935651 , 0.2912874 , 0.28897023, 0.2866218 ,
       0.28425008, 0.2818629 , 0.2794676 , 0.2770714 , 0.27468